In [1]:
#import proximityhash as pr
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.window import Window
import pandas as pd
# import boto3
# import datetime as dt
from datetime import datetime, timedelta
# import proximityhash as pr
# import pygeohash as pgh
# import geojson
# import geohashlite
import json

In [2]:
spark = SparkSession.builder.appName("insights").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/05 12:30:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
geojson_file_path = "/Users/nlblr135/Downloads/usa_county.geojson"

# Load the .geojson file
with open(geojson_file_path, 'r') as f:
    data = json.load(f)

# Extract features from the GeoJSON data
features = data["features"]
rows = [
    {
        "GEOID": feature["properties"]["GEOID"],
        "NAME": feature["properties"]["NAME"],
        "NAMELSAD": feature["properties"]["NAMELSAD"],
        "STATE_NAME": feature["properties"]["STATE_NAME"],
        "geometry": f"MultiPolygon {feature['geometry']['coordinates']}"
    }
    for feature in features
]

# Define the schema for the DataFrame
schema = StructType([
    StructField("GEOID", StringType(), True),
    StructField("NAME", StringType(), True),
    StructField("NAMELSAD", StringType(), True),
    StructField("STATE_NAME", StringType(), True),
    StructField("geometry", StringType(), True)
])

# Create DataFrame
df = spark.createDataFrame(rows, schema)

In [4]:
df.show(5)

24/09/05 12:31:11 WARN TaskSetManager: Stage 0 contains a task of very large size (9871 KiB). The maximum recommended task size is 1000 KiB.


+-----+-------+--------------+----------+--------------------+
|GEOID|   NAME|      NAMELSAD|STATE_NAME|            geometry|
+-----+-------+--------------+----------+--------------------+
|01069|Houston|Houston County|   Alabama|MultiPolygon [[[[...|
|01023|Choctaw|Choctaw County|   Alabama|MultiPolygon [[[[...|
|01005|Barbour|Barbour County|   Alabama|MultiPolygon [[[[...|
|01107|Pickens|Pickens County|   Alabama|MultiPolygon [[[[...|
|01033|Colbert|Colbert County|   Alabama|MultiPolygon [[[[...|
+-----+-------+--------------+----------+--------------------+
only showing top 5 rows



In [5]:
df = df.select("GEOID", "NAMELSAD", "STATE_NAME", "geometry")
df = df.withColumnRenamed("NAMELSAD", "County")
df.show(5)

24/09/05 12:31:13 WARN TaskSetManager: Stage 1 contains a task of very large size (9871 KiB). The maximum recommended task size is 1000 KiB.


+-----+--------------+----------+--------------------+
|GEOID|        County|STATE_NAME|            geometry|
+-----+--------------+----------+--------------------+
|01069|Houston County|   Alabama|MultiPolygon [[[[...|
|01023|Choctaw County|   Alabama|MultiPolygon [[[[...|
|01005|Barbour County|   Alabama|MultiPolygon [[[[...|
|01107|Pickens County|   Alabama|MultiPolygon [[[[...|
|01033|Colbert County|   Alabama|MultiPolygon [[[[...|
+-----+--------------+----------+--------------------+
only showing top 5 rows



In [6]:
df = df.withColumn("Name_county", lower(regexp_replace(col("County"), " ", ""))).withColumn("STATE_NAME_temp", lower(regexp_replace(col("STATE_NAME"), " ", "")))
df.show(5)

24/09/05 12:31:14 WARN TaskSetManager: Stage 2 contains a task of very large size (9871 KiB). The maximum recommended task size is 1000 KiB.


+-----+--------------+----------+--------------------+-------------+---------------+
|GEOID|        County|STATE_NAME|            geometry|  Name_county|STATE_NAME_temp|
+-----+--------------+----------+--------------------+-------------+---------------+
|01069|Houston County|   Alabama|MultiPolygon [[[[...|houstoncounty|        alabama|
|01023|Choctaw County|   Alabama|MultiPolygon [[[[...|choctawcounty|        alabama|
|01005|Barbour County|   Alabama|MultiPolygon [[[[...|barbourcounty|        alabama|
|01107|Pickens County|   Alabama|MultiPolygon [[[[...|pickenscounty|        alabama|
|01033|Colbert County|   Alabama|MultiPolygon [[[[...|colbertcounty|        alabama|
+-----+--------------+----------+--------------------+-------------+---------------+
only showing top 5 rows



In [25]:
texas_data = spark.read.csv("/Users/nlblr135/Downloads/insights/final/county_wise_cel_counts.csv", header = True)
texas_data.show(5)

+----------------+-----+--------------+----------------+----+
|          County|State|Reach Estimate|Percent Visitors|rank|
+----------------+-----+--------------+----------------+----+
|    Brown County|   WI|         28520|          14.0 %|   1|
|Outagamie County|   WI|          8969|           4.0 %|   2|
|Milwaukee County|   WI|          8775|           4.0 %|   3|
|     Cook County|   IL|          7664|           4.0 %|   4|
| Waukesha County|   WI|          6765|           3.0 %|   5|
+----------------+-----+--------------+----------------+----+
only showing top 5 rows



In [8]:
texas_data = texas_data.withColumn("County_temp", lower(regexp_replace(col("County"), " ", ""))).withColumn("State_temp", lower(regexp_replace(col("State"), " ", "")))
texas_data.show(5)

+--------------+-----+--------------+----+-------------+----------+
|        County|State|Reach Estimate|rank|  County_temp|State_temp|
+--------------+-----+--------------+----+-------------+----------+
| Harris County|   TX|        435593|   1| harriscounty|        tx|
|Tarrant County|   TX|        319556|   2|tarrantcounty|        tx|
| Dallas County|   TX|        293879|   3| dallascounty|        tx|
|  Bexar County|   TX|        230875|   4|  bexarcounty|        tx|
| Collin County|   TX|        158077|   5| collincounty|        tx|
+--------------+-----+--------------+----+-------------+----------+
only showing top 5 rows



In [11]:
fibs = spark.read.csv("/Users/nlblr135/Downloads/insights/FIPS_code.csv", header = True)
# fibs = fibs.select("State_name", "State_code")
fibs = fibs.withColumnRenamed("State_name", "State_name_fibs")
fibs.show(5)

+---------------+----------+---------------+---------+
|State_name_fibs|State_code|    County_name|FIPS_code|
+---------------+----------+---------------+---------+
|       New York|        NY|   Bronx County|    36005|
|      Minnesota|        MN|  Dakota County|    27037|
|       Colorado|        CO|    Weld County|    08123|
| North Carolina|        NC|Caldwell County|    37027|
|           Iowa|        IA|    Linn County|    19113|
+---------------+----------+---------------+---------+
only showing top 5 rows



In [13]:
fibs = fibs.withColumn("State_name_temp", lower(regexp_replace(col("State_name_fibs"), " ", ""))).withColumn("County_name_temp", lower(regexp_replace(col("County_name"), " ", "")))
fibs.show(5)


+---------------+----------+---------------+---------+---------------+----------------+
|State_name_fibs|State_code|    County_name|FIPS_code|State_name_temp|County_name_temp|
+---------------+----------+---------------+---------+---------------+----------------+
|       New York|        NY|   Bronx County|    36005|        newyork|     bronxcounty|
|      Minnesota|        MN|  Dakota County|    27037|      minnesota|    dakotacounty|
|       Colorado|        CO|    Weld County|    08123|       colorado|      weldcounty|
| North Carolina|        NC|Caldwell County|    37027|  northcarolina|  caldwellcounty|
|           Iowa|        IA|    Linn County|    19113|           iowa|      linncounty|
+---------------+----------+---------------+---------+---------------+----------------+
only showing top 5 rows



In [14]:
final_df = df.join(fibs, (df.Name_county == fibs.County_name_temp) & (df.STATE_NAME_temp == fibs.State_name_temp), how = "inner")
final_df.show(5)

24/09/05 12:39:49 WARN TaskSetManager: Stage 10 contains a task of very large size (9871 KiB). The maximum recommended task size is 1000 KiB.


+-----+--------------+----------+--------------------+-------------+---------------+---------------+----------+--------------+---------+---------------+----------------+
|GEOID|        County|STATE_NAME|            geometry|  Name_county|STATE_NAME_temp|State_name_fibs|State_code|   County_name|FIPS_code|State_name_temp|County_name_temp|
+-----+--------------+----------+--------------------+-------------+---------------+---------------+----------+--------------+---------+---------------+----------------+
|01069|Houston County|   Alabama|MultiPolygon [[[[...|houstoncounty|        alabama|        Alabama|        AL|Houston County|    01069|        alabama|   houstoncounty|
|01023|Choctaw County|   Alabama|MultiPolygon [[[[...|choctawcounty|        alabama|        Alabama|        AL|Choctaw County|    01023|        alabama|   choctawcounty|
|01005|Barbour County|   Alabama|MultiPolygon [[[[...|barbourcounty|        alabama|        Alabama|        AL|Barbour County|    01005|        alabam

In [16]:
final_df = final_df.select("STATE_NAME", "State_code", "County", "GEOID", "geometry")
final_df.show(5)

24/09/05 12:43:21 WARN TaskSetManager: Stage 12 contains a task of very large size (9871 KiB). The maximum recommended task size is 1000 KiB.


+----------+----------+--------------+-----+--------------------+
|STATE_NAME|State_code|        County|GEOID|            geometry|
+----------+----------+--------------+-----+--------------------+
|   Alabama|        AL|Houston County|01069|MultiPolygon [[[[...|
|   Alabama|        AL|Choctaw County|01023|MultiPolygon [[[[...|
|   Alabama|        AL|Barbour County|01005|MultiPolygon [[[[...|
|   Alabama|        AL|Pickens County|01107|MultiPolygon [[[[...|
|   Alabama|        AL|Colbert County|01033|MultiPolygon [[[[...|
+----------+----------+--------------+-----+--------------------+
only showing top 5 rows



In [17]:
final_df = final_df.withColumnRenamed("County", "County_new").withColumn("County_temp", lower(regexp_replace(col("County_new"), " ", ""))).withColumn("State_code_temp", lower(regexp_replace(col("State_code"), " ", "")))
final_df.show(5)

24/09/05 12:43:22 WARN TaskSetManager: Stage 14 contains a task of very large size (9871 KiB). The maximum recommended task size is 1000 KiB.


+----------+----------+--------------+-----+--------------------+-------------+---------------+
|STATE_NAME|State_code|    County_new|GEOID|            geometry|  County_temp|State_code_temp|
+----------+----------+--------------+-----+--------------------+-------------+---------------+
|   Alabama|        AL|Houston County|01069|MultiPolygon [[[[...|houstoncounty|             al|
|   Alabama|        AL|Choctaw County|01023|MultiPolygon [[[[...|choctawcounty|             al|
|   Alabama|        AL|Barbour County|01005|MultiPolygon [[[[...|barbourcounty|             al|
|   Alabama|        AL|Pickens County|01107|MultiPolygon [[[[...|pickenscounty|             al|
|   Alabama|        AL|Colbert County|01033|MultiPolygon [[[[...|colbertcounty|             al|
+----------+----------+--------------+-----+--------------------+-------------+---------------+
only showing top 5 rows



In [19]:
data = final_df.join(texas_data, (final_df.County_temp == texas_data.County_temp) & (final_df.State_code_temp == texas_data.State_temp))
data.show(5)

24/09/05 12:43:43 WARN TaskSetManager: Stage 17 contains a task of very large size (9871 KiB). The maximum recommended task size is 1000 KiB.


+----------+----------+--------------+-----+--------------------+-------------+---------------+--------------+-----+--------------+----+-------------+----------+
|STATE_NAME|State_code|    County_new|GEOID|            geometry|  County_temp|State_code_temp|        County|State|Reach Estimate|rank|  County_temp|State_temp|
+----------+----------+--------------+-----+--------------------+-------------+---------------+--------------+-----+--------------+----+-------------+----------+
|     Texas|        TX|Kleberg County|48273|MultiPolygon [[[[...|klebergcounty|             tx|Kleberg County|   TX|          5443|  98|klebergcounty|        tx|
|     Texas|        TX|Refugio County|48391|MultiPolygon [[[[...|refugiocounty|             tx|Refugio County|   TX|          1225| 173|refugiocounty|        tx|
|     Texas|        TX| Harris County|48201|MultiPolygon [[[[...| harriscounty|             tx| Harris County|   TX|        435593|   1| harriscounty|        tx|
|     Texas|        TX| Harr

In [20]:
data = data.select("STATE_NAME", "State_code", "County", "Reach Estimate", "rank", "GEOID", "geometry")
data.show()

24/09/05 12:44:22 WARN TaskSetManager: Stage 20 contains a task of very large size (9871 KiB). The maximum recommended task size is 1000 KiB.


+----------+----------+--------------+--------------+----+-----+--------------------+
|STATE_NAME|State_code|        County|Reach Estimate|rank|GEOID|            geometry|
+----------+----------+--------------+--------------+----+-----+--------------------+
|     Texas|        TX|Kleberg County|          5443|  98|48273|MultiPolygon [[[[...|
|     Texas|        TX|Refugio County|          1225| 173|48391|MultiPolygon [[[[...|
|     Texas|        TX| Harris County|        435593|   1|48201|MultiPolygon [[[[...|
|     Texas|        TX| Harris County|        435593|   1|48201|MultiPolygon [[[[...|
|     Texas|        TX| Harris County|        435593|   1|48201|MultiPolygon [[[[...|
|     Texas|        TX| Harris County|        435593|   1|48201|MultiPolygon [[[[...|
|     Texas|        TX| Harris County|        435593|   1|48201|MultiPolygon [[[[...|
|     Texas|        TX| Harris County|        435593|   1|48201|MultiPolygon [[[[...|
|     Texas|        TX| Harris County|        435593| 

In [21]:
data = data.dropDuplicates()

In [23]:
data.coalesce(1).write.csv("/Users/nlblr135/Downloads/insights/geoid_data/new_full/", header = True)

24/09/05 12:45:14 WARN TaskSetManager: Stage 26 contains a task of very large size (9871 KiB). The maximum recommended task size is 1000 KiB.


In [26]:
county_data = spark.read.csv("/Users/nlblr135/Downloads/insights/final/county_wise_cel_counts.csv", header = True)
county_data.show(5)

+----------------+-----+--------------+----------------+----+
|          County|State|Reach Estimate|Percent Visitors|rank|
+----------------+-----+--------------+----------------+----+
|    Brown County|   WI|         28520|          14.0 %|   1|
|Outagamie County|   WI|          8969|           4.0 %|   2|
|Milwaukee County|   WI|          8775|           4.0 %|   3|
|     Cook County|   IL|          7664|           4.0 %|   4|
| Waukesha County|   WI|          6765|           3.0 %|   5|
+----------------+-----+--------------+----------------+----+
only showing top 5 rows



In [27]:
county_data = county_data.withColumn("County_temp", lower(regexp_replace(col("County"), " ", ""))).withColumn("State_temp", lower(regexp_replace(col("State"), " ", "")))
county_data.show(5)

+----------------+-----+--------------+----------------+----+---------------+----------+
|          County|State|Reach Estimate|Percent Visitors|rank|    County_temp|State_temp|
+----------------+-----+--------------+----------------+----+---------------+----------+
|    Brown County|   WI|         28520|          14.0 %|   1|    browncounty|        wi|
|Outagamie County|   WI|          8969|           4.0 %|   2|outagamiecounty|        wi|
|Milwaukee County|   WI|          8775|           4.0 %|   3|milwaukeecounty|        wi|
|     Cook County|   IL|          7664|           4.0 %|   4|     cookcounty|        il|
| Waukesha County|   WI|          6765|           3.0 %|   5| waukeshacounty|        wi|
+----------------+-----+--------------+----------------+----+---------------+----------+
only showing top 5 rows



In [28]:
data = final_df.join(county_data, (final_df.County_temp == county_data.County_temp) & (final_df.State_code_temp == county_data.State_temp))
data.show(5)

24/09/05 19:13:07 WARN TaskSetManager: Stage 38 contains a task of very large size (9871 KiB). The maximum recommended task size is 1000 KiB.


+----------+----------+------------------+-----+--------------------+----------------+---------------+------------------+-----+--------------+----------------+----+----------------+----------+
|STATE_NAME|State_code|        County_new|GEOID|            geometry|     County_temp|State_code_temp|            County|State|Reach Estimate|Percent Visitors|rank|     County_temp|State_temp|
+----------+----------+------------------+-----+--------------------+----------------+---------------+------------------+-----+--------------+----------------+----+----------------+----------+
|   Alabama|        AL|    Houston County|01069|MultiPolygon [[[[...|   houstoncounty|             al|    Houston County|   AL|             1|           0.0 %|2136|   houstoncounty|        al|
|   Alabama|        AL|    Colbert County|01033|MultiPolygon [[[[...|   colbertcounty|             al|    Colbert County|   AL|             3|           0.0 %|1329|   colbertcounty|        al|
|   Arizona|        AZ|     La Paz 

In [30]:
data = data.select("STATE_NAME", "State_code", "County", "Reach Estimate", "Percent Visitors", "GEOID", "geometry")
data.show()

24/09/05 19:14:10 WARN TaskSetManager: Stage 41 contains a task of very large size (9871 KiB). The maximum recommended task size is 1000 KiB.


+----------+----------+------------------+--------------+----------------+-----+--------------------+
|STATE_NAME|State_code|            County|Reach Estimate|Percent Visitors|GEOID|            geometry|
+----------+----------+------------------+--------------+----------------+-----+--------------------+
|   Alabama|        AL|    Houston County|             1|           0.0 %|01069|MultiPolygon [[[[...|
|   Alabama|        AL|    Colbert County|             3|           0.0 %|01033|MultiPolygon [[[[...|
|   Arizona|        AZ|     La Paz County|             2|           0.0 %|04012|MultiPolygon [[[[...|
|  Arkansas|        AR|   Randolph County|             1|           0.0 %|05121|MultiPolygon [[[[...|
|California|        CA|Los Angeles County|           124|           0.0 %|06037|MultiPolygon [[[[...|
|California|        CA|Los Angeles County|           124|           0.0 %|06037|MultiPolygon [[[[...|
|California|        CA|Los Angeles County|           124|           0.0 %|06037|Mu

In [31]:
data = data.dropDuplicates()

In [32]:
data.coalesce(1).write.csv("/Users/nlblr135/Downloads/insights/geoid_data/new_full_county/", header = True)

24/09/05 19:16:01 WARN TaskSetManager: Stage 44 contains a task of very large size (9871 KiB). The maximum recommended task size is 1000 KiB.


In [9]:
final_df = final_df.withColumnRenamed("County", "County_new").withColumn("County_temp", lower(regexp_replace(col("County_new"), " ", ""))).withColumn("State_code_temp", lower(regexp_replace(col("State_code"), " ", "")))
final_df.show(5)


NameError: name 'final_df' is not defined

In [ ]:
data = data.select("STATE_NAME", "State_code", "County", "Reach Estimate", "rank", "GEOID", "geometry")
data.show()

In [ ]:
data = data.dropDuplicates()

In [ ]:
data.coalesce(1).write.csv("/Users/nlblr135/Downloads/insights/geoid_data/new/", header = True)

In [ ]:
county_data = spark.read.csv("/Users/nlblr135/Downloads/insights/final_county_raw_data/county_count_raw.csv", header = True)
county_data.show(5)

In [ ]:
final_df = final_df.select("STATE_NAME", "State_code", "County", "GEOID", "geometry")
final_df.show(5)

In [ ]:
final_df = final_df.withColumnRenamed("County", "County_new").withColumn("County_temp", lower(regexp_replace(col("County_new"), " ", ""))).withColumn("State_code_temp", lower(regexp_replace(col("State_code"), " ", "")))
final_df.show(5)

county_data = county_data.withColumn("County_temp", lower(regexp_replace(col("County"), " ", ""))).withColumn("State_temp", lower(regexp_replace(col("State"), " ", "")))
county_data.show(5)




In [ ]:
data = final_df.join(county_data, (final_df.County_temp == county_data.County_temp) & (final_df.State_code_temp == county_data.State_temp))
data.show(5)

In [ ]:
data = data.select("STATE_NAME", "State_code", "County", "count", "percentage", "GEOID", "geometry")
data.show()

In [ ]:
data = data.dropDuplicates()

In [ ]:
data.orderBy(col("percentage").desc()).show(5)

In [ ]:
data.coalesce(1).write.csv("/Users/nlblr135/Downloads/insights/geoid_data/", header = True)

In [ ]:
fibs = fibs.withColumn("State_name_temp", lower(regexp_replace(col("State_name_fibs"), " ", ""))).withColumn("County_name_temp", lower(regexp_replace(col("County_name"), " ", "")))
fibs.show(5)


In [ ]:
final_df = df.join(fibs, ())


In [ ]:
county_data.orderBy(col("count").desc()).show(5)

In [ ]:
import requests
import json


url = "http://int-apigw.azira.com/estimation-api/v5/analytics/audience/build/estimate-reach/footfalls?apikey=XZ1Oz2RmpVVGp8UuGDkq5p7f1OKE7i41&scale=true&frequency=1&tenant_id=2cb3bb59&tp=30&frequency_ope$"

payload = json.dumps({
  "country": "USA",
  "tenant_id": "2cb3bb59",
  "ctv": False,
  "locations": {
    "or_bucket": {
      "admin_regions": [
        "admin_area_lv2,AL2840-2370695"
      ]
    }
  },
  "metadata_type": "has_point"
})
headers = {
  'Content-Type': 'application/json'
}


response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

In [ ]:
spark = SparkSession.builder.appName("dummy").getOrCreate()

In [ ]:
df = spark.read.csv("/Users/nlblr135/Downloads/door_county_cel.tsv",sep = "\t", header = True)

In [ ]:
df.select("Common Evening Admin").distinct().count()

In [ ]:
df.show(20, False)

In [ ]:
df.select("admin_area_lv2").distinct().count()

In [ ]:
df_new = df.withColumn('State Code', split(col('admin_area_lv2'), ', ')[1])

# Create a new 'County' column which contains only the county name
df_new = df_new.withColumn('admin_area_lv2', split(col('admin_area_lv2'), ', ')[0])

# Create the 'FIPS code' column by extracting the first 5 digits of 'admin_area_lv6'
df_new = df_new.withColumn('FIPS code', substring(col('admin_area_lv6'), 1, 5))

In [ ]:
df_new.show(20, False)

In [ ]:
df_new = df_new.withColumnRenamed("admin_area_lv2", "County_name")


In [ ]:
df_new = df_new.select("State_name", "State_code", "County_name", "FIPS_code")

In [ ]:
df_new = df_new.dropDuplicates()

In [ ]:
df_new.distinct().count()

In [ ]:
df_new.coalesce(1).write.csv("/Users/nlblr135/Downloads/insights/FIPS/", header = True)